In [ ]:
import rasterio
import h5py as hdf
import numpy as np
from rasterio.windows import Window
from rasterio.crs import CRS


In [ ]:
# set the addres of HDF file
inputHdfile = r"address.extension"

# set the address for the output raster.
writeRaster = r"address.extension"

In [ ]:
# this is the projection that the hdf has been built upon
src_profile ={'driver': 'HFA', 'dtype': 'uint8', 'nodata': 255.0, 'width': 161229, 'height': 104481, 'count': 1, 'crs': CRS.from_wkt('PROJCS["Albers_Conic_Equal_Area",GEOGCS["GCS_GRS_1980_IUGG_1980",DATUM["D_unknown",SPHEROID["GRS80",6378137,298.257222101]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.3],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'), 'transform': rasterio.Affine(30.0, 0.0, -2493641.627346346,
       0.0, -30.0, 3310674.817445671), 'blockxsize': 64, 'blockysize': 64, 'tiled': True}

In [ ]:
src_profile

In [ ]:
# find the key(s) of the hdf file
with hdf.File(inputHdfile, 'r') as f:
    key = f.keys()
    print(f.keys())

In [ ]:
# set the key
key='ST'

In [ ]:
# you can set the datatype or the format of the file here. HFA is the .img format.
src_profile['driver']= 'HFA'
src_profile['dtype'] = 'float64'

In [ ]:
w_raster =  src_profile['width']
h_raster = src_profile['height']

In [ ]:
# Window(col_off,row_off,width,height)
lstWins =[
    (0,0,w_raster, 20000),
    (0,20000,w_raster,20000),
    (0,40000,w_raster,20000),
    (0,60000,w_raster,20000),
    (0,80000,w_raster,h_raster-80000)
]
lstStEnd =[
    #rowStart,rowEnd,colStart,colEnd
    [0,20000,0,w_raster],
    [20000,40000,0,w_raster],
    [40000,60000,0,w_raster],
    [60000,80000,0,w_raster],
    [80000,h_raster,0,w_raster]
]

with rasterio.open(writeRaster, 'w', **src_profile) as dst:
    print(dst.shape)
    with hdf.File(inputHdfile,'r') as f:
        arData = f[key]
        for win,stEnd in zip(lstWins, lstStEnd):
            print (win,stEnd)
            Z = arData[stEnd[0]:stEnd[1],stEnd[2]:stEnd[3]]
            print(Z.shape)
            dst.write(Z,window=Window(win[0],win[1],win[2],win[3]), indexes = 1)